# **Get new CSV Data from NASA**
Area of interest: Madagascar

## From the web
### Step 1:
Access this link : https://firms.modaps.eosdis.nasa.gov/download/
### Step 2:
- Enter your email
- You'll get your FIRMS login code on your email
- Once you are authenticated on the NASA FIRMS interface, create new request
- In our case :
 - choose "country" from the dropdown list, and select "Madagascar"
 - check all the MODIS and VIIRS fire source
 - enter the date range
 - choose CSV from the last dropddown list
- And you wait for NASA to send your download link  

## From the API
Let's see the full list of available data

In [1]:
import polars as pl
MAP_KEY='35ae49800327f79d4046cf4fd1f86349'

In [2]:
data_availability = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pl.read_csv(data_availability)
display(df)

data_id,min_date,max_date
str,str,str
"""MODIS_NRT""","""2025-04-01""","""2025-06-22"""
"""MODIS_SP""","""2000-11-01""","""2025-03-31"""
"""VIIRS_NOAA20_NRT""","""2025-03-01""","""2025-06-22"""
"""VIIRS_NOAA20_SP""","""2018-04-01""","""2025-02-28"""
"""VIIRS_NOAA21_NRT""","""2024-01-17""","""2025-06-22"""
…,…,…
"""VIIRS_SNPP_SP""","""2012-01-20""","""2025-02-28"""
"""LANDSAT_NRT""","""2022-06-20""","""2025-06-21"""
"""GOES_NRT""","""2022-08-09""","""2025-06-22"""


We will use data_id values like VIIRS_* and MODIS_*

Note: You enter a start_date, and the API gives data for the next 10 days

In [3]:
start_date = '2025-04-27'     # enter the start_date here, format : yyyy-mm-dd

modis_nrt = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_NRT/MDG/10/' + start_date
df_modis_nrt = pl.read_csv(modis_nrt)

modis_sp = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_SP/MDG/10/' + start_date
df_modis_sp=pl.read_csv(modis_sp)

We'll put our new data in Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


We compute the end_date (10 days after the start_date)

In [5]:
from datetime import datetime, timedelta

def add_10_days(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    new_date = date_obj + timedelta(days=10)
    return new_date.strftime("%Y-%m-%d")

end_date=add_10_days(start_date)

We create folders for our new data

In [6]:
import os
folder_path = "/content/drive/MyDrive/LIAM/Fire_Project/AI_project/data/raw/csv_NASA/"
new_modis_folder = folder_path + 'modis/modis_' + start_date + '_' + end_date
os.makedirs(folder_path + 'modis/modis_' + start_date + '_' + end_date, exist_ok=True)

# add check if start_date between folders_name

We download CSVs

In [11]:
if not df_modis_nrt.is_empty():
    df_modis_nrt.write_csv(new_modis_folder + "/modis_nrt.csv")
else:
    print("df_modis_nrt is empty, no CSV downloaded.")

if not df_modis_sp.is_empty():
    df_modis_sp.write_csv(new_modis_folder + "/modis_sp.csv")
else:
    print("df_modis_sp is empty, no CSV downloaded.")

print("CSV files have been downloaded.")

df_modis_sp is empty, no CSV downloaded.
CSV files have been downloaded.
